In [23]:
team_id = '40'
split = "test_1"

In [24]:
from src.dataloader import base_loader

df, df_test = base_loader(split)

In [25]:
from src.embedding import base_embedder
from sklearn import preprocessing

X = base_embedder(df.words_str.values)
y_reg = df.score_compound.values
enc = preprocessing.LabelEncoder()
y_clf = enc.fit_transform(df.sentiment.values)

X_test = base_embedder(df_test.words_str.values)

### Regression

In [26]:
import pandas as pd
import smogn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train, X_val, y_train, y_val = train_test_split(X, y_reg, test_size=0.2, random_state=42, stratify=y_clf)
# temp = pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name="y_train")], axis=1)

# temp_smogn = smogn.smoter(data = temp, y = "y_train")

# y_resampled = temp_smogn["y_train"]
# X_resampled = temp_smogn.drop["y_train"]

X_resampled = X_train
y_resampled = y_train

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [27]:
from sklearn.svm import SVR

reg = SVR().fit(X_resampled, y_resampled)

In [28]:
from src.evaluation import eval_reg

eval_reg(reg, datasets)


Split: train
	RMSE: 0.21
	MAE: 0.15

Split: valididation
	RMSE: 0.31
	MAE: 0.24


In [29]:
from src.utils import save_sub

save_sub(model=reg, X_test=X_test, split=split, task='reg')

(1000,)
[ 0.03587885 -0.59095088  0.33802556  0.52473458]


### Classification

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, y_clf, test_size=0.2, random_state=42, stratify=y_clf)

X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [31]:
from sklearn.svm import SVC

clf = SVC().fit(X_resampled, y_resampled)

In [32]:
from src.evaluation import eval_clf

eval_clf(clf, datasets)


Split: train
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4232
           1       0.92      0.89      0.91      4232
           2       0.93      0.91      0.92      4232

    accuracy                           0.93     12696
   macro avg       0.93      0.93      0.93     12696
weighted avg       0.93      0.93      0.93     12696


Split: validation
              precision    recall  f1-score   support

           0       0.44      0.55      0.49        96
           1       0.82      0.83      0.83      1058
           2       0.69      0.63      0.66       446

    accuracy                           0.76      1600
   macro avg       0.65      0.67      0.66      1600
weighted avg       0.76      0.76      0.76      1600



In [33]:
from src.utils import save_sub

save_sub(model=clf, X_test=X_test, split=split, enc=enc, task='clf')

(1000,)
['neutral' 'neutral' 'neutral' 'positive']


### Submission

#### Regression

In [34]:
reg = SVR().fit(X, y_reg)

In [37]:
eval_reg(reg, [[X, y_reg], [X, y_reg]])


Split: train
	RMSE: 0.21
	MAE: 0.15

Split: valididation
	RMSE: 0.21
	MAE: 0.15


In [38]:
save_sub(model=reg, X_test=X_test, split=split, task='reg')

(1000,)
[-0.05459659 -0.53004469  0.33235456  0.54861968]


#### Classification

In [39]:
X_rs, y_rs = SMOTE().fit_resample(X, y_clf)

In [40]:
clf = SVC().fit(X_rs, y_rs)

In [41]:
eval_clf(clf, [[X, y_clf], [X, y_clf]])


Split: train
              precision    recall  f1-score   support

           0       0.67      0.94      0.78       482
           1       0.92      0.90      0.91      5290
           2       0.83      0.82      0.83      2228

    accuracy                           0.88      8000
   macro avg       0.81      0.88      0.84      8000
weighted avg       0.88      0.88      0.88      8000


Split: validation
              precision    recall  f1-score   support

           0       0.67      0.94      0.78       482
           1       0.92      0.90      0.91      5290
           2       0.83      0.82      0.83      2228

    accuracy                           0.88      8000
   macro avg       0.81      0.88      0.84      8000
weighted avg       0.88      0.88      0.88      8000



In [42]:
save_sub(model=clf, X_test=X_test, split=split, enc=enc, task='clf')

(1000,)
['neutral' 'neutral' 'positive' 'positive']
